# Coastal Currents Explorer

This notebook allows you to interactively explore current data for the Darwin/Beagle Gulf region using H3 hexagons and PixiJS vector visualization.

## Workflow:
1. Define an Area of Interest (AOI) bounding box on a map
2. Extract and visualize coastal landmass data within the AOI
3. Load and visualize Copernicus current data as vector fields using H3 hexagons
4. Explore data with adjustable resolution and interactive controls

In [ ]:
# Import necessary libraries
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import Draw, MeasureControl
import xarray as xr
import h3
import json
import tempfile
from IPython.display import display, HTML, IFrame
from ipywidgets import interact, widgets, Layout, Button, HBox, VBox
from shapely.geometry import Polygon, Point, box

## 1. Define paths to data

First, let's set up the paths to our data files.

In [ ]:
# Set up paths
# Add parent directory to sys.path
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(os.getcwd()))))

# Define paths to data
COASTAL_DATA_PATH = os.path.abspath(os.path.join(
    os.path.dirname(os.getcwd()), 
    'data/CoastalWatersAustraliaAndOceania/polygons_CoastalWatersAustraliaAndOceania.parquet'
))

CURRENT_DATA_PATH = os.path.abspath(os.path.join(
    os.path.dirname(os.getcwd()), 
    '_tmp/ntoz.zarr'  # Update to the actual path if different
))

# Check if files exist
print(f"Coastal data file exists: {os.path.exists(COASTAL_DATA_PATH)}")
print(f"Current data file exists: {os.path.exists(CURRENT_DATA_PATH)}")

# Define default AOI around Darwin
DEFAULT_BBOX = [130.5, -13.0, 131.2, -12.0]  # [min_lon, min_lat, max_lon, max_lat]

## 2. Interactive Area Selection

Let's create a map interface to select our Area of Interest (AOI).

In [ ]:
def create_aoi_selection_map(center=[-12.46, 130.84], zoom_start=8):
    """Create an interactive map centered on Darwin with drawing tools for AOI selection"""
    m = folium.Map(location=center, zoom_start=zoom_start, tiles='CartoDB positron')
    
    # Add drawing tools
    draw = Draw(
        export=True,
        position='topleft',
        draw_options={
            'rectangle': True,  # Only enable rectangle for BBOX selection
            'polyline': False,
            'polygon': False,
            'circle': False,
            'marker': False,
            'circlemarker': False
        }
    )
    draw.add_to(m)
    
    # Add measurement tools
    MeasureControl(position='bottomleft').add_to(m)
    
    # Add title and instructions
    title_html = '''
        <div style="position: fixed; top: 10px; left: 50px; width: 300px; 
                    background-color: white; padding: 10px; z-index: 9999; border-radius: 5px;">
            <h3 style="margin: 0;">Define Area of Interest</h3>
            <p style="margin: 5px 0 0 0;">Draw a rectangle in the Darwin/Timor Sea region</p>
            <p>Click the rectangle icon and drag on the map</p>
        </div>
    '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    return m

def extract_bbox_from_geojson(geojson_data):
    """Extract bounding box coordinates from the drawn rectangle"""
    try:
        # Extract coordinates from the first feature (should be our rectangle)
        feature = geojson_data['features'][0]
        if feature['geometry']['type'] != 'Polygon':
            return None
        
        # Get coordinates
        coords = feature['geometry']['coordinates'][0]
        
        # Calculate bbox [min_lon, min_lat, max_lon, max_lat]
        min_lon = min(coord[0] for coord in coords)
        min_lat = min(coord[1] for coord in coords)
        max_lon = max(coord[0] for coord in coords)
        max_lat = max(coord[1] for coord in coords)
        
        return [min_lon, min_lat, max_lon, max_lat]
    except (KeyError, IndexError, TypeError) as e:
        print(f"Error extracting bbox: {e}")
        return None

In [ ]:
# Create selection map
selection_map = create_aoi_selection_map()
display(selection_map)

### Manual BBOX Input

Since we can't easily capture the drawn rectangle programmatically in Jupyter, let's set up a manual input form for the bounding box coordinates.

In [ ]:
# Create widgets for manual BBOX input
min_lon_widget = widgets.FloatText(value=DEFAULT_BBOX[0], description='Min Lon:', step=0.1)
min_lat_widget = widgets.FloatText(value=DEFAULT_BBOX[1], description='Min Lat:', step=0.1)
max_lon_widget = widgets.FloatText(value=DEFAULT_BBOX[2], description='Max Lon:', step=0.1)
max_lat_widget = widgets.FloatText(value=DEFAULT_BBOX[3], description='Max Lat:', step=0.1)

bbox_button = widgets.Button(description='Set BBOX')

# Set global variable to store the selected bbox
selected_bbox = None

def on_bbox_button_click(b):
    global selected_bbox
    selected_bbox = [
        min_lon_widget.value, min_lat_widget.value, 
        max_lon_widget.value, max_lat_widget.value
    ]
    print(f"BBOX set to: {selected_bbox}")

bbox_button.on_click(on_bbox_button_click)

display(widgets.HBox([min_lon_widget, min_lat_widget, max_lon_widget, max_lat_widget, bbox_button]))

## 3. Coastal Data Processing

Now let's load and process the coastal data from the parquet file.

In [ ]:
def load_coastal_data(parquet_path):
    """Load coastal data from parquet file"""
    print(f"Loading coastal data from: {parquet_path}")
    gdf = gpd.read_parquet(parquet_path)
    
    # Ensure CRS is set correctly
    if gdf.crs is None:
        gdf.set_crs(epsg=4326, inplace=True)
    
    print(f"Loaded {len(gdf)} features with CRS: {gdf.crs}")
    return gdf

def extract_coastal_data_for_bbox(gdf, bbox):
    """Extract coastal data for the specified bounding box"""
    # Create a shapely box
    bbox_poly = box(bbox[0], bbox[1], bbox[2], bbox[3])
    
    # Filter to features that intersect the bbox
    bbox_gdf = gdf[gdf.geometry.intersects(bbox_poly)].copy()
    
    # Simplify geometries for better performance (optional)
    bbox_gdf['geometry'] = bbox_gdf.geometry.simplify(tolerance=0.001)
    
    print(f"Filtered to {len(bbox_gdf)} coastal features in the BBOX")
    return bbox_gdf

def visualize_coastal_data(bbox_gdf, bbox):
    """Create a map visualization of coastal data within the bbox"""
    # Calculate center of bbox
    center_lon = (bbox[0] + bbox[2]) / 2
    center_lat = (bbox[1] + bbox[3]) / 2
    
    # Create map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=9,
        tiles='CartoDB positron'
    )
    
    # Add coastal features
    folium.GeoJson(
        bbox_gdf,
        name='Coastal Features',
        style_function=lambda x: {
            'fillColor': '#0096c7',
            'color': '#023e8a',
            'weight': 1,
            'fillOpacity': 0.6
        }
    ).add_to(m)
    
    # Add bbox outline
    folium.Rectangle(
        bounds=[(bbox[1], bbox[0]), (bbox[3], bbox[2])],
        color='red',
        weight=2,
        fill=False
    ).add_to(m)
    
    # Add title
    title_html = f'''
        <div style="position: fixed; top: 10px; left: 50px; width: 250px; 
                    background-color: white; padding: 10px; z-index: 9999; border-radius: 5px;">
            <h3 style="margin: 0;">Coastal Features</h3>
            <p style="margin: 5px 0 0 0;">{len(bbox_gdf)} features in selected area</p>
        </div>
    '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    folium.LayerControl().add_to(m)
    return m

In [ ]:
# Load full coastal dataset
full_coastal_gdf = load_coastal_data(COASTAL_DATA_PATH)

In [ ]:
# Extract and visualize coastal data for the selected bbox
if selected_bbox is None:
    print("Please set a BBOX first using the form above.")
else:
    coastal_gdf = extract_coastal_data_for_bbox(full_coastal_gdf, selected_bbox)
    coastal_map = visualize_coastal_data(coastal_gdf, selected_bbox)
    display(coastal_map)

## 4. Copernicus Current Data Processing

Now let's load and process the current data from the zarr file.

In [ ]:
def load_copernicus_current_data(zarr_path, bbox=None, time_index=0):
    """Load Copernicus current data from zarr file"""
    print(f"Loading current data from: {zarr_path}")
    
    try:
        # Open zarr dataset
        ds = xr.open_zarr(zarr_path)
        
        # Basic info
        print(f"Dataset dimensions: {list(ds.dims.items())}")
        print(f"Variables: {list(ds.data_vars)}")
        
        # Subset by bbox if provided
        if bbox:
            ds = ds.sel(
                longitude=slice(bbox[0], bbox[2]),
                latitude=slice(bbox[3], bbox[1])  # Note reversed order for descending lat
            )
        
        # Subset by time 
        if 'time' in ds.dims and time_index is not None:
            ds = ds.isel(time=time_index)
            print(f"Selected time index: {time_index}")
        
        return ds
    except Exception as e:
        print(f"Error loading current data: {e}")
        return None

In [ ]:
# Load current data
if selected_bbox is None:
    print("Please set a BBOX first using the form above.")
else:
    current_ds = load_copernicus_current_data(CURRENT_DATA_PATH, selected_bbox)
    
    # Display sample data if available
    if current_ds is not None:
        print("\nSample current data:")
        display(current_ds)

## 5. H3 Hexagon Grid Creation

Now let's create an H3 hexagon grid for our area and calculate current vectors for each hexagon.

In [ ]:
def create_h3_grid(bbox, resolution=7):
    """Create an H3 hexagon grid covering the bounding box"""
    # Create a polygon from the bbox
    polygon = Polygon([
        (bbox[0], bbox[1]),  # bottom-left
        (bbox[0], bbox[3]),  # top-left
        (bbox[2], bbox[3]),  # top-right
        (bbox[2], bbox[1]),  # bottom-right
        (bbox[0], bbox[1])   # close polygon
    ])
    
    # Get hexagons covering the polygon
    h3_indices = h3.polyfill(polygon.__geo_interface__, resolution)
    print(f"Created {len(h3_indices)} H3 hexagons at resolution {resolution}")
    
    return h3_indices

def calculate_currents_for_h3_grid(ds, h3_indices):
    """Calculate average current vectors for each H3 hexagon"""
    result = []
    
    # Get u and v components
    u_var = 'uo'  # Eastward current
    v_var = 'vo'  # Northward current
    
    # Get data arrays
    u = ds[u_var].values
    v = ds[v_var].values
    
    # Get the coordinate arrays
    lons = ds.longitude.values
    lats = ds.latitude.values
    
    # Create coordinate meshgrid for faster lookups
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    
    # For each hexagon
    for h3_idx in h3_indices:
        # Get hexagon center
        center_lat, center_lon = h3.h3_to_geo(h3_idx)
        
        # Get hexagon boundary
        boundary = h3.h3_to_geo_boundary(h3_idx)
        hex_poly = Polygon(boundary)
        
        # Find all grid points within this hexagon
        u_values = []
        v_values = []
        
        # For computational efficiency, first check if any points are in the hexagon's bounding box
        min_lon = min(p[1] for p in boundary)
        max_lon = max(p[1] for p in boundary)
        min_lat = min(p[0] for p in boundary)
        max_lat = max(p[0] for p in boundary)
        
        # Find grid points within hexagon (simplified approach)
        points_in_bbox = np.where(
            (lon_grid >= min_lon) & (lon_grid <= max_lon) &
            (lat_grid >= min_lat) & (lat_grid <= max_lat)
        )
        
        for i, j in zip(points_in_bbox[0], points_in_bbox[1]):
            if i < len(lats) and j < len(lons):
                point = Point(lon_grid[i, j], lat_grid[i, j])
                if hex_poly.contains(point):
                    if not np.isnan(u[i, j]) and not np.isnan(v[i, j]):
                        u_values.append(u[i, j])
                        v_values.append(v[i, j])
        
        # Calculate average current if we found any valid points
        if u_values and v_values:
            avg_u = np.mean(u_values)
            avg_v = np.mean(v_values)
            magnitude = np.sqrt(avg_u**2 + avg_v**2)
            
            result.append({
                'h3_index': h3_idx,
                'center_lat': center_lat,
                'center_lon': center_lon,
                'u': avg_u,
                'v': avg_v,
                'magnitude': magnitude
            })
    
    return pd.DataFrame(result)

In [ ]:
# Create resolution selection widget
def create_resolution_widget():
    """Create a widget to adjust H3 resolution"""
    resolution_slider = widgets.IntSlider(
        value=7,
        min=5,
        max=9,
        step=1,
        description='H3 Resolution:',
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        layout=Layout(width='50%')
    )
    
    # Display info about resolution
    resolution_info = {
        5: "~8 km edge length - Low detail, fast",
        6: "~4 km edge length - Medium-low detail",
        7: "~2 km edge length - Medium detail (default)",
        8: "~1 km edge length - High detail",
        9: "~500 m edge length - Very high detail, slow"
    }
    
    help_text = widgets.HTML(
        value=f"<p><b>Current setting:</b> {resolution_info[resolution_slider.value]}</p>"
    )
    
    def update_help_text(change):
        help_text.value = f"<p><b>Current setting:</b> {resolution_info[change['new']]}</p>"
    
    resolution_slider.observe(update_help_text, names='value')
    
    return widgets.VBox([resolution_slider, help_text]), resolution_slider

# Display resolution widget
resolution_box, resolution_slider = create_resolution_widget()
display(resolution_box)

In [ ]:
# Calculate current vectors for H3 grid
if 'current_ds' not in globals() or current_ds is None:
    print("Please load current data first by setting a BBOX and running the previous cell.")
else:
    # Create H3 grid and calculate currents
    h3_indices = create_h3_grid(selected_bbox, resolution=resolution_slider.value)
    currents_df = calculate_currents_for_h3_grid(current_ds, h3_indices)
    
    # Display sample of currents data
    print(f"Calculated currents for {len(currents_df)} hexagons")
    display(currents_df.head())

## 6. Folium-based Visualization

Let's create a simple Folium-based visualization first.

In [ ]:
def get_color_for_magnitude(magnitude):
    """Return a color based on current magnitude"""
    if magnitude < 0.2:
        return '#2c7bb6'  # Light blue
    elif magnitude < 0.5:
        return '#abd9e9'  # Medium blue
    elif magnitude < 1.0:
        return '#fdae61'  # Orange
    else:
        return '#d7191c'  # Red

def visualize_currents_h3(currents_df, bbox, coastal_gdf=None):
    """Create a map visualization of current vectors on H3 hexagons"""
    # Calculate center
    center_lon = (bbox[0] + bbox[2]) / 2
    center_lat = (bbox[1] + bbox[3]) / 2
    
    # Create map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=9,
        tiles='CartoDB positron'
    )
    
    # Add coastal features if provided
    if coastal_gdf is not None:
        folium.GeoJson(
            coastal_gdf,
            name='Coastal Features',
            style_function=lambda x: {
                'fillColor': '#0096c7',
                'color': '#023e8a',
                'weight': 1,
                'fillOpacity': 0.4
            }
        ).add_to(m)
    
    # Add H3 hexagons with current vectors
    for _, row in currents_df.iterrows():
        # Get hexagon boundary
        boundary = h3.h3_to_geo_boundary(row['h3_index'])
        
        # Create hexagon
        folium.Polygon(
            locations=boundary,
            color='gray',
            weight=1,
            fill=True,
            fill_opacity=0.1
        ).add_to(m)
        
        # Add current vector arrow
        scale_factor = 0.02  # Adjust based on typical magnitude values
        arrow_length = row['magnitude'] * scale_factor
        
        # Calculate arrow endpoint
        end_lat = row['center_lat'] + row['v'] * arrow_length
        end_lon = row['center_lon'] + row['u'] * arrow_length
        
        # Color based on magnitude
        color = get_color_for_magnitude(row['magnitude'])
        
        # Create arrow
        folium.PolyLine(
            locations=[(row['center_lat'], row['center_lon']), (end_lat, end_lon)],
            color=color,
            weight=2,
            arrow_style='->', # Arrow display may need tweaking
            popup=f"U: {row['u']:.2f}, V: {row['v']:.2f}, Mag: {row['magnitude']:.2f} m/s"
        ).add_to(m)
    
    # Add legend
    legend_html = '''
    <div style="position: fixed; bottom: 50px; right: 50px; width: 180px; 
                background-color: white; padding: 10px; z-index: 9999; border-radius: 5px;">
        <h4 style="margin: 0;">Current Speed (m/s)</h4>
        <div><span style="background-color: #2c7bb6; width: 15px; height: 15px; display: inline-block;"></span> &lt; 0.2</div>
        <div><span style="background-color: #abd9e9; width: 15px; height: 15px; display: inline-block;"></span> 0.2 - 0.5</div>
        <div><span style="background-color: #fdae61; width: 15px; height: 15px; display: inline-block;"></span> 0.5 - 1.0</div>
        <div><span style="background-color: #d7191c; width: 15px; height: 15px; display: inline-block;"></span> &gt; 1.0</div>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Add title
    title_html = f'''
        <div style="position: fixed; top: 10px; left: 50px; width: 250px; 
                    background-color: white; padding: 10px; z-index: 9999; border-radius: 5px;">
            <h3 style="margin: 0;">Ocean Currents</h3>
            <p style="margin: 5px 0 0 0;">{len(currents_df)} hexagons with current data</p>
        </div>
    '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    folium.LayerControl().add_to(m)
    return m

In [ ]:
# Visualize currents with Folium
if 'currents_df' not in globals() or currents_df is None:
    print("Please calculate currents first by running the previous cell.")
else:
    folium_map = visualize_currents_h3(currents_df, selected_bbox, coastal_gdf)
    display(folium_map)

## 7. PixiJS Visualization

Now let's create a more advanced visualization using PixiJS for better performance.

In [ ]:
def create_pixijs_template(currents_df, bbox, coastal_geojson=None, width=800, height=600):
    """Create HTML/JS template for PixiJS-based current vector visualization"""
    # Convert currents data to JSON for JavaScript
    currents_json = currents_df.to_json(orient='records')
    
    # Create bounding box in format expected by JavaScript
    js_bbox = {
        'west': bbox[0],
        'south': bbox[1],
        'east': bbox[2],
        'north': bbox[3]
    }
    
    # Create HTML template with embedded PixiJS
    html_template = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <title>Current Vectors Visualization</title>
        <style>
            #visualization-container {{
                width: {width}px;
                height: {height}px;
                position: relative;
                margin: 0 auto;
            }}
            .control-panel {{
                position: absolute;
                top: 10px;
                right: 10px;
                background-color: rgba(255, 255, 255, 0.8);
                padding: 10px;
                border-radius: 5px;
                z-index: 1000;
            }}
            .legend {{
                position: absolute;
                bottom: 20px;
                right: 10px;
                background-color: rgba(255, 255, 255, 0.8);
                padding: 10px;
                border-radius: 5px;
                z-index: 1000;
            }}
            .legend-item {{
                display: flex;
                align-items: center;
                margin: 5px 0;
            }}
            .legend-color {{
                width: 20px;
                height: 5px;
                margin-right: 5px;
            }}
        </style>
        <!-- Load PixiJS -->
        <script src="https://cdn.jsdelivr.net/npm/pixi.js@6.5.2/dist/browser/pixi.min.js"></script>
        <!-- Load Leaflet for base map -->
        <link rel="stylesheet" href="https://unpkg.com/leaflet@1.7.1/dist/leaflet.css" />
        <script src="https://unpkg.com/leaflet@1.7.1/dist/leaflet.js"></script>
    </head>
    <body>
        <div id="visualization-container">
            <div id="map" style="width: 100%; height: 100%;"></div>
            
            <div class="control-panel">
                <div>
                    <label for="vector-scale">Vector Scale:</label>
                    <input type="range" id="vector-scale" min="0.1" max="5" step="0.1" value="1.0">
                    <span id="scale-value">1.0</span>
                </div>
                <div>
                    <label for="vector-density">Vector Density:</label>
                    <input type="range" id="vector-density" min="0.2" max="1.0" step="0.1" value="1.0">
                    <span id="density-value">1.0</span>
                </div>
            </div>
            
            <div class="legend">
                <h4 style="margin-top: 0;">Current Speed (m/s)</h4>
                <div class="legend-item">
                    <div class="legend-color" style="background-color: #2c7bb6;"></div>
                    &lt; 0.2
                </div>
                <div class="legend-item">
                    <div class="legend-color" style="background-color: #abd9e9;"></div>
                    0.2 - 0.5
                </div>
                <div class="legend-item">
                    <div class="legend-color" style="background-color: #fdae61;"></div>
                    0.5 - 1.0
                </div>
                <div class="legend-item">
                    <div class="legend-color" style="background-color: #d7191c;"></div>
                    &gt; 1.0
                </div>
            </div>
        </div>
        
        <script>
            // Current data from Python
            const currentsData = {currents_json};
            const bbox = {js_bbox};
            
            // Set up Leaflet map
            const map = L.map('map').setView(
                [(bbox.north + bbox.south) / 2, (bbox.east + bbox.west) / 2], 
                10
            );
            
            L.tileLayer('https://{{s}}.tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png', {{
                attribution: '&copy; OpenStreetMap contributors'
            }}).addTo(map);
            
            // Fit map to bbox
            map.fitBounds([
                [bbox.south, bbox.west],
                [bbox.north, bbox.east]
            ]);
            
            // Add coastal features if provided
            const coastalData = {coastal_geojson if coastal_geojson else 'null'};
            if (coastalData) {{
                L.geoJSON(coastalData, {{
                    style: {{
                        fillColor: '#0096c7',
                        color: '#023e8a',
                        weight: 1,
                        fillOpacity: 0.4
                    }}
                }}).addTo(map);
            }}
            
            // Add PixiJS overlay for vectors
            const pixiContainer = document.createElement('div');
            pixiContainer.style.position = 'absolute';
            pixiContainer.style.top = '0';
            pixiContainer.style.left = '0';
            pixiContainer.style.pointerEvents = 'none';
            map._container.appendChild(pixiContainer);
            
            // Initialize PixiJS
            const app = new PIXI.Application({{
                width: map._container.clientWidth,
                height: map._container.clientHeight,
                transparent: true,
                antialias: true
            }});
            pixiContainer.appendChild(app.view);
            
            // Vector layer
            const vectorLayer = new PIXI.Container();
            app.stage.addChild(vectorLayer);
            
            // Colors for different magnitudes
            function getColorForMagnitude(magnitude) {{
                if (magnitude < 0.2) return 0x2c7bb6;
                if (magnitude < 0.5) return 0xabd9e9;
                if (magnitude < 1.0) return 0xfdae61;
                return 0xd7191c;
            }}
            
            // Vector scaling and density controls
            let vectorScale = 1.0;
            let vectorDensity = 1.0;
            
            document.getElementById('vector-scale').addEventListener('input', function(e) {{
                vectorScale = parseFloat(e.target.value);
                document.getElementById('scale-value').textContent = vectorScale.toFixed(1);
                updateVectors();
            }});
            
            document.getElementById('vector-density').addEventListener('input', function(e) {{
                vectorDensity = parseFloat(e.target.value);
                document.getElementById('density-value').textContent = vectorDensity.toFixed(1);
                updateVectors();
            }});
            
            // Function to update vector visualization
            function updateVectors() {{
                // Clear previous vectors
                vectorLayer.removeChildren();
                
                // Choose subset of vectors based on density
                const displayVectors = currentsData.filter(() => Math.random() < vectorDensity);
                
                // Draw vectors
                for (const vector of displayVectors) {{
                    // Calculate screen coordinates
                    const point = map.latLngToContainerPoint([vector.center_lat, vector.center_lon]);
                    
                    // Draw arrow
                    const arrow = new PIXI.Graphics();
                    const baseLength = vector.magnitude * 20 * vectorScale;
                    const arrowLength = Math.max(5, baseLength);  // Minimum size
                    
                    // Calculate arrow endpoint
                    const angle = Math.atan2(vector.v, vector.u);
                    const endX = arrowLength * Math.cos(angle);
                    const endY = arrowLength * Math.sin(angle);
                    
                    // Arrow properties
                    const arrowColor = getColorForMagnitude(vector.magnitude);
                    const arrowWidth = Math.min(3, 1 + vector.magnitude);
                    
                    // Draw line
                    arrow.lineStyle(arrowWidth, arrowColor, 1);
                    arrow.moveTo(0, 0);
                    arrow.lineTo(endX, endY);
                    
                    // Draw arrowhead
                    const headLength = Math.min(7, arrowLength * 0.3);
                    const headAngle = Math.PI / 6;  // 30 degrees
                    
                    arrow.lineTo(
                        endX - headLength * Math.cos(angle - headAngle),
                        endY - headLength * Math.sin(angle - headAngle)
                    );
                    arrow.moveTo(endX, endY);
                    arrow.lineTo(
                        endX - headLength * Math.cos(angle + headAngle),
                        endY - headLength * Math.sin(angle + headAngle)
                    );
                    
                    // Position arrow
                    arrow.x = point.x;
                    arrow.y = point.y;
                    
                    vectorLayer.addChild(arrow);
                }}
            }}
            
            // Update vectors when map moves
            map.on('moveend', function() {{
                updateVectors();
            }});
            
            // Initial vector update
            updateVectors();
            
            // Handle window resize
            window.addEventListener('resize', function() {{
                app.renderer.resize(map._container.clientWidth, map._container.clientHeight);
                updateVectors();
            }});
        </script>
    </body>
    </html>
    """
    
    return html_template

def visualize_currents_pixijs(currents_df, bbox, coastal_gdf=None, width=800, height=600):
    """Create a PixiJS-based visualization of current vectors"""
    # Convert coastal GeoDataFrame to GeoJSON if provided
    coastal_geojson = None
    if coastal_gdf is not None:
        coastal_geojson = json.dumps(coastal_gdf.__geo_interface__)
    
    # Create HTML template with PixiJS
    html = create_pixijs_template(currents_df, bbox, coastal_geojson, width, height)
    
    # Save HTML to a temporary file
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "currents_visualization.html")
    
    with open(temp_path, "w") as f:
        f.write(html)
    
    # Display using IFrame
    return IFrame(src=temp_path, width=width, height=height)

In [ ]:
# Visualize currents with PixiJS
if 'currents_df' not in globals() or currents_df is None:
    print("Please calculate currents first by running the previous cells.")
else:
    print("Rendering current data with PixiJS (high-performance visualization)")
    pixijs_viz = visualize_currents_pixijs(currents_df, selected_bbox, coastal_gdf, width=900, height=700)
    display(pixijs_viz)

## 8. Resolution Control and Update

Let's add interactive control to update the visualization with different H3 resolutions.

In [ ]:
def update_visualization(resolution):
    if 'current_ds' not in globals() or current_ds is None or selected_bbox is None:
        print("Please load data first by setting a BBOX and running the previous cells.")
        return
    
    # Create H3 grid and calculate currents
    print(f"Creating H3 grid with resolution {resolution}...")
    h3_indices = create_h3_grid(selected_bbox, resolution=resolution)
    currents_df = calculate_currents_for_h3_grid(current_ds, h3_indices)
    
    # Create PixiJS visualization
    print(f"Rendering visualization with {len(currents_df)} hexagons...")
    pixijs_viz = visualize_currents_pixijs(currents_df, selected_bbox, coastal_gdf, width=900, height=700)
    display(pixijs_viz)

# Create button to update visualization
update_button = widgets.Button(
    description='Update Visualization',
    button_style='primary',
    tooltip='Update with current resolution'
)

def on_update_button_click(b):
    update_visualization(resolution_slider.value)

update_button.on_click(on_update_button_click)

# Display button
display(update_button)

## 9. Challenges and Next Steps

This notebook demonstrates an interactive visualization of current data using H3 hexagons and PixiJS. However, there are several challenges and potential improvements:

### Current Challenges

1. **Performance Constraints**:
   - High-resolution H3 grids (resolution 8-9) may be slow to generate for large areas
   - Current data interpolation uses a simple approach that could be improved

2. **Technical Limitations**:
   - PixiJS integration requires file I/O for the HTML output
   - Folium and PixiJS require separate implementations
   - No time dimension handling for current patterns over time

### Next Steps

1. **Improve Data Integration**:
   - Add support for multiple time points in the Copernicus data
   - Implement proper spatial interpolation for current values
   - Add depth dimension exploration

2. **Enhance Visualization**:
   - Add streamline visualization for continuous flow representation
   - Implement interactive site selection on the map
   - Add export capabilities for processed data

3. **Mission Planning Features**:
   - Integrate with USV power calculations
   - Add route optimization between survey sites
   - Calculate station-keeping power requirements